In [120]:
import torch
from stable_baselines3 import PPO
import gymnasium as gym
import numpy as np
import imageio
import subprocess
import shutil
import time
import os

In [96]:
class CustomRewardWrapper(gym.Wrapper):
    def __init__(self, env):
        super(CustomRewardWrapper, self).__init__(env)

    def step(self, action):
        obs, reward, done, truncated, info = self.env.step(action)
        # Access relevant state and action variables
        
        #state
        z = obs[0]                  # Height of the robot
        a = obs[1]                  # Angle of the robot
        a_hip = obs[2]              # Angle of the hip
        a_knee = obs[3]             # Angle of the knee
        a_ankle = obs[4]            # Angle of the ankle
        v_x = obs[5]                # Velocity in x direction
        v_z = obs[6]                # Velocity in z direction
        a_d = obs[7]                # Angular velocity 
        a_hip_d = obs[8]            # Angular velocity of the hip
        a_knee_d = obs[9]           # Angular velocity of the knee
        a_ankle_d = obs[10]         # Angular velocity of the ankle
        
        #action
        torque_hip = action[0]      # Torque applied to the hip
        torque_knee = action[1]     # Torque applied to the knee
        torque_ankle = action[2]    # Torque applied to the ankle

        #different criteria for reward
        energy_used = np.sum(np.square(action))  # Simplistic energy calculation

        # Custom reward logic
        custom_reward = np.tanh(a) * 0.5                           # Reward for positive angles
        custom_reward += 5*np.exp(-(a-(2*np.pi))**2/(2*np.pi))      # Reward for 2pi angle
        custom_reward -= energy_used * 0.1                          # Penalize energy consumption

        # used by openai
        # backroll = -obs[7]
        # height = obs[0]
        # vel_act = action[0] * obs[8] + action[1] * obs[9] + a[2] * obs[10]
        # backslide = -obs[5]
        # reward = backroll * (1.0 + .3 * height + .1 * vel_act + .05 * backslide)

        if done:
            custom_reward -= 8  # Heavy penalty for falling

        return obs, custom_reward, done, truncated, info

In [121]:
healthy_reward = 0.8
healthy_z_range = (0.2, float("inf"))
healthy_angle_range = (-np.pi/4, 2*np.pi+np.pi/4)
reset_noise_scale = 5e-3
exclude_current_positions_from_observation = True

env = gym.make('Hopper-v4', render_mode='rgb_array', healthy_reward=healthy_reward, healthy_z_range=healthy_z_range, healthy_angle_range=healthy_angle_range, reset_noise_scale=reset_noise_scale, exclude_current_positions_from_observation=exclude_current_positions_from_observation)
env = CustomRewardWrapper(env)

model = PPO("MlpPolicy", env, verbose=1)

# Train the model
n_learning_steps = 30000
model.learn(total_timesteps=n_learning_steps)

# Save the model
model.save("hopper")

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 43.6     |
|    ep_rew_mean     | -20.5    |
| time/              |          |
|    fps             | 1051     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 56.3        |
|    ep_rew_mean          | -21.9       |
| time/                   |             |
|    fps                  | 573         |
|    iterations           | 2           |
|    time_elapsed         | 7           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.010734132 |
|    clip_fraction        | 0.103       |
|    clip_range           | 0.2         |
|    entropy_loss   

In [116]:
vec_env = model.get_env()
obs = vec_env.reset()

writer = imageio.get_writer('hopper-flip.mp4', fps=100)

N_step = 100000
for i in range(N_step):
    action, _state = model.predict(obs, deterministic=True)
    obs, reward, done, info = vec_env.step(action)
    print(obs[0][1])
    writer.append_data(vec_env.render("rgb_array"))
    #VecEnv resets automatically
    if done:
      print("Episode finished after {} timesteps".format(i+1))
      break
      obs = vec_env.reset()

writer.close()

# d = time.strftime("%Y-%m-%d-%H-%M-%S")
# current_py_file = os.getcwd() + '/hopper.ipynb'
# new_py_file = os.getcwd() + '/Run/hopper%s.ipynb'%(d)

# shutil.copyfile(current_py_file, new_py_file)
# shutil.copyfile('hopper-flip.mp4', os.getcwd()+'/Render/hopper-flip%s.mp4'%(d))

0.002712728331846149
0.003643348925547453
0.005009833826853255
0.00669379838812032
0.008519238191886751
0.010150073683339244
0.012126839992988393
0.015466671364019892
0.020319317846429676
0.02682366905210253
0.03506038534687783
0.04509949772585869
0.05678373432564986
0.06996311245834766
0.08473488394667411
0.10050663630803938
0.11635951184434361
0.13235464459695637
0.14829309880288255
0.1616785263647102
0.17341569054727
0.18434811869630574
0.19494202406142122
0.205442842027757
0.21597003671781342
0.22657342031107447
0.23726602676353553
0.24804289831330079
0.25889149926581756
0.2697972350538747
0.2807461680100874
0.2917261723882001
0.3027272700483812
0.31374213293786163
0.3247661334394311
0.33579536378729874
0.346825847620062
0.3578536924092507
0.3688754174066022
0.37988814913621605
0.39088962010053685
0.4018780209154528
0.41285178513997833
0.4238093698860712
0.43474906651957174
0.4456688523274343
0.45656627879305156
0.4674383873586353
0.47828165475098755
0.4890919627009706
0.4998645903

In [119]:
np.pi/2

1.5707963267948966